# Introduzione

Utilizzare il conda environment **trascrizione_audio**

Questa sezione è dedicata a modelli per la trascrizione di audio in testo.    
Tuttavia spesso è utile trasformare il contenuto di un video in testo, per fare ciò occorre quindi trasformare i video in audio.    
Questo script mira proprio a questo, ovvero alla trasformazione **video** in **audio**.   

### Utilizzare il pacchetto **pydub** per trasformare video in file audio.

In [4]:
import os # Muoversi tra le directory
from pydub import AudioSegment # Trasforma video in audio
from tqdm import tqdm # Barra di progresso

# Pacchetti per parallelizzare 
import concurrent.futures
import multiprocessing

In [5]:
# Path della cartella nella quale sono i file da convertire
path = "/Users/mattia/Desktop/Camera cafe/Quinta stagione/"

# Path per specificare dove salvare gli audio  che verranno convertiti
path_destination_audio= "/Users/mattia/Desktop/Università/Data Science in Python/13) Neural Networks/Modelli per Trascrizione Audio/"

# Andare alla cartella  nella quale ci sono i file da convertire
os.chdir(path)

# Ottenere una lista di tutti i file nella cartella, così si avrà una lista dei file da convertire
puntate = os.listdir()

In [6]:
# Funzione per convertire un singolo file
def convert_video_to_audio(video_file : str,
                           path:str,
                           path_destination_audio : str
                          ):
    """Questa funzione prende  un file video  tramite il path e lo trasforma in un audio.
       Va specificato il nome del file, il path nella directory dove si trova, il path della directory dove inserire il file audio risultante. 
    """
    try:
        # Ignora file che non sono video
        if not video_file.lower().endswith(('.mp4', '.mkv', '.avi', '.mov', '.wmv', '.flv')):
            return None
            
        # Percorso completo del file video
        video_path = os.path.join(path, video_file)
        
        # Nome del file audio di output
        audio_filename = os.path.splitext(video_file)[0] + ".mp3"
        audio_path = os.path.join(path_destination_audio, audio_filename)
        
        # Convertire il video in audio
        audio = AudioSegment.from_file(video_path)
        
        # Salva l'audio estratto come MP3
        audio.export(audio_path, format="mp3")
        
        print(f"✅ Convertito in: {audio_filename}")
        return None
    except Exception as e:
        return (video_file, str(e))

In [7]:
# Calcola il numero di processi da utilizzare (lasciando un core libero)
max_workers = max(1, multiprocessing.cpu_count() - 1)
print(f"Utilizzo di {max_workers} core per la conversione (1 core lasciato libero)")

file_problemi = []

Utilizzo di 7 core per la conversione (1 core lasciato libero)


In [ ]:
# Utilizzo del ThreadPoolExecutor per la parallelizzazione
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Prepara i job per l'esecuzione parallela
    futures = {executor.submit(convert_video_to_audio,
                                    video_file=file,
                                    path_destination_audio=path_destination_audio,
                                    path=path): file for file in puntate}
    
    # Monitora il progresso man mano che i job vengono completati
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
        file_name = futures[future]
        try:
            result = future.result()
            if result:  # Se c'è stato un errore
                file_problemi.append(result)
                print(f"❌ Errore per il file: {result[0]} - {result[1]}")
        except Exception as e:
            file_problemi.append((file_name, str(e)))
            print(f"❌ Errore per il file: {file_name} - {str(e)}")

100%|██████████| 179/179 [00:00<00:00, 112224.28it/s]

❌ Errore per il file: Ep. 27 - Viaggio premio.mp4 - convert_video_to_audio() missing 2 required positional arguments: 'path' and 'path_destination_audio'
❌ Errore per il file: Ep. 53 - Te lo dico in faccia.mp4 - convert_video_to_audio() missing 2 required positional arguments: 'path' and 'path_destination_audio'
❌ Errore per il file: Ep. 157 - Nuda proprieta.mp4 - convert_video_to_audio() missing 2 required positional arguments: 'path' and 'path_destination_audio'
❌ Errore per il file: Ep. 42 - Pensiero profondo.mp4 - convert_video_to_audio() missing 2 required positional arguments: 'path' and 'path_destination_audio'
❌ Errore per il file: Ep. 139 - Segnale anti-gaffe.mp4 - convert_video_to_audio() missing 2 required positional arguments: 'path' and 'path_destination_audio'
❌ Errore per il file: Ep. 169 - Voglie.mp4 - convert_video_to_audio() missing 2 required positional arguments: 'path' and 'path_destination_audio'
❌ Errore per il file: Ep. 68 - Paolo testimone.mp4 - convert_video_t